In [68]:
import numpy as np
import plotly.graph_objects as go
from scipy.io import wavfile
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

def importWAV(filename):
    samplerate, rawData = wavfile.read(filename)
    
    time = np.linspace(0, rawData.shape[0]/samplerate, rawData.shape[0])   
    
    data = {'left':rawData[:, 0],'right':rawData[:, 1]}
    return time,data

In [69]:
v_500 = importWAV('40mV_500Hz.wav')
v_1k = importWAV('40mV_1kHz.wav')
v_2k = importWAV('40mV_2kHz.wav')
v_4k = importWAV('40mV_4kHz.wav')
v_8k = importWAV('40mV_8kHz.wav')

t_500 = v_500[0]
t_1k = v_1k[0]
t_2k = v_2k[0]
t_4k = v_4k[0]
t_8k = v_8k[0]

t_arr = np.array([t_500, t_1k, t_2k, t_4k, t_8k])

v_500 = v_500[1]['right']
v_1k = v_1k[1]['right']
v_2k = v_2k[1]['right']
v_4k = v_4k[1]['right']
v_8k = v_8k[1]['right']

v_arr = np.array([v_500, v_1k, v_2k, v_4k, v_8k])

f_arr = np.array([5E2, 1E3, 2E3, 4E3, 8E3])

/var/folders/rg/hh4cb_xd5_74c2r0hymqq4jm0000gn/T/ipykernel_6065/1836492366.py:8: WavFileWarning:

Chunk (non-data) not understood, skipping it.

/var/folders/rg/hh4cb_xd5_74c2r0hymqq4jm0000gn/T/ipykernel_6065/2693771237.py:13: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.

/var/folders/rg/hh4cb_xd5_74c2r0hymqq4jm0000gn/T/ipykernel_6065/2693771237.py:21: VisibleDeprecationWarning:

Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.



In [70]:
global f_c

V_amp = 28.25E-3 * np.sqrt(2)
V_amp_std = 0.02E-3

def fit_sin(t, a, b):
    global f_c
    return a * V_amp * np.sin(f_c * (2*np.pi) * t + b)

C_arr = np.array([])
C_std_arr = np.array([])

for v, f in zip(v_arr, f_arr):
    n = round(7 * f - 400)
    v_c = np.copy(v)
    v_sorted = np.sort(v_c)
    amps = (v_sorted[(-n - 1): -51] - v_sorted[50: n]) / 2
    C = np.mean(amps) / V_amp
    C_std = np.sqrt(np.std(amps, ddof=1) ** 2 + V_amp_std ** 2)
    
    C_arr = np.append(C_arr, C)
    C_std_arr = np.append(C_std_arr, C_std)

print(C_arr, C_std_arr)

[9.27489218 9.21220154 9.05802091 8.9347134  8.65786157] [1.69163557e-04 1.59061982e-04 8.67176320e-05 6.19791693e-04
 3.13538568e-03]


In [76]:
fig = go.Figure()
fig.add_trace(go.Scatter(
    x=f_arr, y=C_arr,
    name='Calculated C',
    type='markers'
))

fig.update_layout(
    xaxis_title='Frequency (Hz)',
    yaxis_title='Calibration Constant',
    xaxis=dict(
        type='log'
    )
)

ValueError:     The 'type' property of scatter is read-only